In [15]:
import json
import stanza
import pickle
import datetime
import time
import re
from multiprocessing import Process, Manager, Pool
import os, time, random
import pymongo 
from pymongo import MongoClient

In [16]:
client = MongoClient(username="hald",password="wzx10221126")

In [17]:
now=datetime.datetime.now()

In [18]:
now_time=now.strftime('%Y_%m_%d')

In [19]:
now_time='2022_12_09'

In [20]:
pairs=['Gene','RNA','Carbohydrate','Lipid','Peptide', 'Pharmaceutical_Preparations', 'Protein', 'Toxin','Mutation', 'Disease']

In [21]:
db = client.hald

In [8]:
Gene_entities=list(db.Gene_raw.find({'addtime':now_time}))
RNA_entities=list(db.RNA_raw.find({'addtime':now_time}))
Carbohydrate_entities=list(db.Carbohydrate_raw.find({'addtime':now_time}))
Lipid_entities=list(db.Lipid_raw.find({'addtime':now_time}))
Peptide_entities=list(db.Peptide_raw.find({'addtime':now_time}))
Pharmaceutical_Preparations_entities=list(db.Pharmaceutical_Preparations_raw.find({'addtime':now_time}))
Protein_entities=list(db.Protein_raw.find({'addtime':now_time}))
Toxin_entities=list(db.Toxin_raw.find({'addtime':now_time}))
Mutation_entities=list(db.Mutation_raw.find({'addtime':now_time}))
Disease_entities=list(db.Disease_raw.find({'addtime':now_time}))
# Mutation_raw_entities=list(db.Mutation_raw.find({'addtime':now_time}))
# Disease_raw_entities=list(db.Disease_raw.find({'addtime':now_time}))

In [9]:
Mutation_all_entities=list(db.Mutation_raw.find())
Disease_all_entities=list(db.Disease_raw.find())

In [10]:
Mutation_dropout_entities={}
Disease_dropout_entities={}
for i in Mutation_all_entities:
    if not i.get('Genomic View'):
        Mutation_dropout_entities[i.get('entity')]=i.get('entity')
for i in Disease_all_entities:
    if not i.get('url'):
        Disease_dropout_entities[i.get('entity')]=i.get('entity')

In [11]:
len(Mutation_dropout_entities)

759

In [12]:
len(Disease_dropout_entities)

13986

In [14]:
len(Disease_entities)

27730

In [17]:
names=locals()

In [18]:
for j in pairs:
    names[j+'_dict']={}
    for i in names[j+'_entities']:
        names[j+'_dict'][i.get('PMID')]=names[j+'_dict'].get(i.get('PMID'),[])+[i]


In [19]:
diff_relation_dict={}
for j in range(len(pairs)-1):
    for i in range(j+1,len(pairs)):
        for k in list(set(names[pairs[j]+'_dict'].keys()).intersection(set(names[pairs[i]+'_dict'].keys()))):
            for l in names[pairs[j]+'_dict'].get(k):
                for m in names[pairs[i]+'_dict'].get(k):
                    if l.get('sentence')==m.get('sentence') and l.get('entity')!=m.get('entity'):
                        diff_relation_dict[l.get('sentence')]=diff_relation_dict.get(l.get('sentence'),[])+[{'PMID':l.get('PMID'),
                                                                                                  'entity1':l.get('entity'),
                                                                                                   'entity2':m.get('entity'),
                                                                                                   'sentence':l.get('sentence'),
                                                                                                   'AB':l.get('AB'),
                                                                                                   'target1':l.get('target'),
                                                                                                    'target1_type':l.get('type'),
                                                                                                   'target2':m.get('target'),
                                                                                                    'target2_type':m.get('type'),
                                                                                                   'PMID':l.get('PMID'),
                                                                                                  }]

In [20]:
same_relation_dict={}
for j in range(len(pairs)):
    for k in names[pairs[j]+'_dict'].values():
        for l in range(len(k)-1):
            for m in range(l+1,len(k)):
                if k[l].get('sentence')==k[m].get('sentence'):
                    if k[l].get('entity')!=k[m].get('entity'):
                        same_relation_dict[k[l].get('sentence')]=same_relation_dict.get(k[l].get('sentence'),[])+[{'PMID':k[l].get('PMID'),
                                                                                                  'entity1':k[l].get('entity'),
                                                                                                   'entity2':k[m].get('entity'),
                                                                                                   'sentence':k[l].get('sentence'),
                                                                                                   'AB':k[l].get('AB'),
                                                                                                   'target1':k[l].get('target'),
                                                                                                    'target1_type':k[l].get('type'),
                                                                                                   'target2':k[m].get('target'),
                                                                                                    'target2_type':k[l].get('type'),
                                                                                                   'PMID':k[l].get('PMID'),
                                                                                                  }]

In [21]:
print(len(same_relation_dict))
print(len(diff_relation_dict))
print(len(list(set(list(same_relation_dict.keys())).intersection(set(list(diff_relation_dict.keys()))))))

4409
1119
417


In [22]:
relation_dict={}
for i in same_relation_dict.keys():
    if diff_relation_dict.get(i):
        relation_dict[i]=same_relation_dict.get(i)+diff_relation_dict.get(i)

In [23]:
for i in same_relation_dict.keys():
    if not relation_dict.get(i):
        relation_dict[i]=same_relation_dict.get(i)
print(len(relation_dict))
for i in diff_relation_dict.keys():
    if not relation_dict.get(i):
        relation_dict[i]=diff_relation_dict.get(i)
print(len(relation_dict))

4409
5111


In [24]:
import os
os.makedirs('./openie/filelist/')

In [25]:
ind=1
for i in relation_dict.keys():
    f=open('./openie/filelist/sentence'+str(ind)+'.txt','w')
    f.write(i)
    f.close()
    ind+=1

In [26]:
f=open('./openie/filelist.txt','a')
for i in range(1,len(relation_dict)+1):
    f.write('filelist/sentence'+str(i)+'.txt'+'\n')
f.close()

In [27]:
len(relation_dict)

5111

In [28]:
sentences=list(relation_dict.keys())

In [29]:
os.makedirs('./openie/files/')
k=1
for i in range(0,len(sentences),len(sentences)//20):
    pickle.dump(sentences[i:i+len(sentences)//20],open('./openie/files/openie'+str(k)+'.pkl','wb'))
    k+=1

In [31]:
os.makedirs('./rules/files/')
k=1
for i in range(0,len(sentences),len(sentences)//20):
    f=open('./rules/files/rules'+str(k)+'.py','w')
    f.write('''
import stanza
import os
import datetime
import time
nlp = stanza.Pipeline('en', package='genia',download_method=None)
import pickle
raw=pickle.load(open('../../openie/files/openie'''+str(k)+'''.pkl','rb'))
import re 

def nlpSentence(i,sentence2words,sentence2relations,sentence2verblist,sentence2verbtype,sentence2adv):
    sentence=i
    doc=nlp(sentence)
    words=[]
    for sent in doc.sentences:
        for word in sent.words:
            words.append(word.text)
    relations=[]
    verblist=[]
    verbprototype=[]
    adv=[]
    for sent in doc.sentences:
        for word in sent.words:
            relations.append([word.text,words[int(word.head)-1],int(word.id),int(word.head),word.deprel,word.xpos])
            if word.xpos in ['VB','VBD','VBG','VBN', 'VBP', 'VBZ']:
                verblist.append({word.text:word.xpos})
                verbprototype.append({word.text:word.lemma})
            if word.xpos in ['RB','RBR','RBS']:
                adv.append({word.text:word.xpos})
    sentence2words[i]=words
    sentence2relations[i]=relations
    sentence2verblist[i]=verblist
    sentence2verbprototype[i]=verbprototype
    sentence2adv[i]=adv


start=datetime.datetime.now()
print('Parent process %s.' % os.getpid())
sentence2words={}
sentence2relations={}
sentence2verblist={}
sentence2verbprototype={}
sentence2adv={}
for i in raw:
    nlpSentence(i,sentence2words,sentence2relations,sentence2verblist,sentence2verbprototype,sentence2adv)
pickle.dump(sentence2words,open('sentence2words'''+str(k)+'''.pkl','wb'))
pickle.dump(sentence2relations,open('sentence2relations'''+str(k)+'''.pkl','wb'))
pickle.dump(sentence2verblist,open('sentence2verblist'''+str(k)+'''.pkl','wb'))
pickle.dump(sentence2verbprototype,open('sentence2verbprototype'''+str(k)+'''.pkl','wb'))
pickle.dump(sentence2adv,open('sentence2adv'''+str(k)+'''.pkl','wb'))
print('Waiting for all subprocesses done...')

end=datetime.datetime.now()
print("The running time is "+str((end-start).seconds)+"s")  
    ''')
    f.close()
    k+=1

In [ ]:
# shell
# cd openie
# nohup java -mx500g -cp ../../stanza_corenlp/stanford-corenlp-4.3.2.jar:../../stanza_corenlp/stanford-corenlp-4.3.2-models.jar:../../stanza_corenlp/CoreNLP-to-HTML.xsl:../../slf4j-api.jar:../../stanza_corenlp/slf4j-simple.jar edu.stanford.nlp.naturalli.OpenIE -format reverb -filelist ./filelist.txt -output ./openie.txt > ./openie.log &

In [46]:
#shell 
# cd rules/files
# for i in {1..21}; do echo "nohup python rules${i}.py > rules.log &" | bash; done

In [32]:
import pandas as pd

In [33]:
openie_table=pd.read_table('openie/openie.txt',header=None)

In [34]:
openie_table.head(3)

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16
0,filelist/sentence4.txt,0,regression statistics,is in,SPSS 16.0,28,30,30,31,31,33,1.0,"Association of SOD1 50 bp Indel , GPX1 ( rs180...","NNP IN NN CD NN NNP , NN -LRB- NN -RRB- , CC N...",regression statistics,be in,SPSS 16.0
1,filelist/sentence4.txt,0,Association genotypes,is with,clinical variables,0,18,18,19,20,22,1.0,"Association of SOD1 50 bp Indel , GPX1 ( rs180...","NNP IN NN CD NN NNP , NN -LRB- NN -RRB- , CC N...",Association genotype,be with,clinical variable
2,filelist/sentence17.txt,0,four CB results,obtained from,carcinoma NOS,1,4,4,6,6,8,1.0,All four CB results obtained from carcinoma NO...,"DT CD NN NNS VBN IN NN NNS VBD JJ IN NNP , NN ...",four cb result,obtain from,carcinoma no


In [57]:
# 3770378
len(openie_table)

NameError: name 'openie_table' is not defined

In [36]:
df=openie_table.loc[:,[0,2,3,4,12]]

In [37]:
sentences=list(relation_dict.keys())

In [38]:
be=['be','am','is','are','was','were','have been','has been']
bedict={}
for i in be:
    bedict[i]='be'


In [39]:
rules_pairs=['sentence2adv','sentence2relations','sentence2verblist','sentence2verbprototype','sentence2words']

In [40]:
for i in rules_pairs:
    names[i]={}
    for k in ['rules/files/'+i+str(j)+'.pkl' for j in range(1,22)]:
        names[i].update(pickle.load(open(k,'rb')))
    # pickle.dump(names[i],open('files/'+i+'.pkl','wb'))

In [28]:
finallist=[]
for line in range(len(df)):
    i=sentences[int(df.iloc[line,0][17:-4])-1]
    verblist=sentence2verblist.get(i)
    verbrelation=''
    real_verb=''
    verbdict={}
    verbs_split=df.loc[line,3].split(' ')
    for verb in verblist:
        verbdict.update(verb)
    # if len(verbs.split(' '))==1:
    #     real_verb=verbs.split(' ')[0]
    # else:

    for indexx,one in enumerate(verbs_split):
        if verbdict.get(one):
            verbrelation=one
            relations=sentence2relations.get(i)
            relations_display=[relation[-2] for relation in relations]
            verbprototype=sentence2verbprototype.get(i)
            verbprototype_dict={}
            for verbp in verbprototype:
                verbprototype_dict.update(verbp)
            for index_passive,passive in enumerate(relations_display):
                if passive=='aux:pass':
                    if relations[index_passive][1]==verbrelation:
                        verb_prototype=verbrelation
                        break
                elif passive=='acl' and relations[index_passive][-1]=='VBN':
                    if relations[index_passive][0]==verbrelation:
                        verb_prototype=verbrelation
                        break
            else:
                for v in verbprototype:
                    if v.get(verbrelation):
                        verb_prototype=v.get(verbrelation)
                        break
            verbs_split[indexx]=verb_prototype
        elif bedict.get(one):
            verbs_split[indexx]=bedict.get(one)
    if bedict.get(' '.join(verbs_split[:2])):
        final=' '.join(['be']+verbs_split[2:])
    else:
        final=' '.join(verbs_split)
    finallist.append(final)
df[17]=finallist

In [29]:
ind=1
for i in relation_dict.keys():
    names['sentence'+str(ind)]=i
    ind+=1

In [30]:
import copy

In [293]:
relation_dict_copy=copy.deepcopy(relation_dict)

In [294]:
import re
suc=[]
for i in range(1,len(relation_dict_copy)+1):
    if relation_dict_copy.get(names[df.iloc[i-1,0][9:-4]]):
        openie=[]
        for j in relation_dict_copy.get(names[df.iloc[i-1,0][9:-4]]):
            j1=j.get('target1').translate(str.maketrans({"-":  r"\-","\\": r"\\", "^":  r"\^","$":  r"\$","*":  r"\*",".":  r"\.","(":  r"\(",")":  r"\)","+":  r"\+","[":  r"\[","]":  r"\]",
                      "{":  r"\{","}":  r"\}","|":  r"\|","?":  r"\?"}))
            j2=j.get('target2').translate(str.maketrans({"-":  r"\-","\\": r"\\", "^":  r"\^","$":  r"\$","*":  r"\*",".":  r"\.","(":  r"\(",")":  r"\)","+":  r"\+","[":  r"\[","]":  r"\]",
                      "{":  r"\{","}":  r"\}","|":  r"\|","?":  r"\?"}))
            if re.search(j1,df.iloc[i-1,1]) and re.search(j2,df.iloc[i-1,3]):
                j['openie']=j.get('openie',[])+[[j.get('target1'),df.iloc[i-1,2],j.get('target2')]]
                openie.append([j.get('target1'),df.iloc[i-1,2],j.get('target2')])
            elif re.search(j2,df.iloc[i-1,1]) and re.search(j1,df.iloc[i-1,3]):
                j['openie']=j.get('openie',[])+[[j.get('target2'),df.iloc[i-1,2],j.get('target1')]]
                openie.append([j.get('target2'),df.iloc[i-1,2],j.get('target1')]) 
        if not openie:
            suc.append(names[df.iloc[i-1,0][9:-4]])
    
        

KeyError: 'sentence4'

In [33]:
error=[]
for i in range(1,len(relation_dict_copy)+1):
    if not relation_dict_copy.get(names[df.iloc[i-1,0][9:-4]]):
        error.append(names[df.iloc[i-1,0][9:-4]])

In [34]:
len(suc)

5846

In [35]:
len(relation_dict_copy)

6741

In [36]:
len(error)

0

In [37]:
import sys
sys.path.append("..")
from allennlp.predictors.predictor import Predictor
import allennlp_models.structured_prediction
PREDICTOR = Predictor.from_path("https://storage.googleapis.com/allennlp-public-models/openie-model.2020.03.26.tar.gz")

In [38]:
import re

def clean(sent):
    return re.sub(' +', ' ', sent.replace('\r', ' ').replace('\n', ' '))


def get_tags_allen(sent):
    sent = sent.replace('[', '(').replace(']', ')')
    res = PREDICTOR.predict(
      sentence=sent
    )
    return res


def parse_allen(ss):
    words = ss['words']
    
    res = []
    for v in ss['verbs']:
        tags = v['tags']

        d = {}
        for i in range(len(words)):
            t = tags[i].replace('I-', '').replace('B-', '')
            w = words[i]
            if t in d:
                d[t] += ' '+ w
            else:
                d[t] = w
        res.append(d)
    return res

In [ ]:
sentence2allennlp={}
for i in relation_dict_copy.keys():
    res=get_tags_allen(i)
    ress=parse_allen(res)
    sentence2allennlp[i]=ress

In [ ]:
import re
for i,j in relation_dict_copy.items():
    for k in j:
        k1=k.get('target1').translate(str.maketrans({"-":  r"\-","\\": r"\\", "^":  r"\^","$":  r"\$","*":  r"\*",".":  r"\.","(":  r"\(",")":  r"\)","+":  r"\+","[":  r"\[","]":  r"\]",
                  "{":  r"\{","}":  r"\}","|":  r"\|","?":  r"\?"}))
        k2=k.get('target2').translate(str.maketrans({"-":  r"\-","\\": r"\\", "^":  r"\^","$":  r"\$","*":  r"\*",".":  r"\.","(":  r"\(",")":  r"\)","+":  r"\+","[":  r"\[","]":  r"\]",
                  "{":  r"\{","}":  r"\}","|":  r"\|","?":  r"\?"}))
        for l in sentence2allennlp.get(i):
            if l.get('ARG1') and l.get('ARG2') and l.get('V'):
                if re.search(k1,l.get('ARG1')) and re.search(k2,l.get('ARG2')):
                    k['allennlp']=k.get('allennlp',[])+[[k.get('target1'), l.get('V') ,k.get('target2')]]
                elif re.search(k2,l.get('ARG1')) and re.search(k1,l.get('ARG2')):
                    k['allennlp']=k.get('allennlp',[])+[[k.get('target2'), l.get('V') ,k.get('target1')]]

In [ ]:
for i,j in relation_dict_copy.items():
    for k in j:
        if k.get('allennlp'):
            print(k)
            break
    else:
        continue
    break

In [ ]:
len(sentence2adv)

In [ ]:
negativewords=['no','not','never','hardly','barely','scarcely','rarely','few','little','seldom','neither','nor','negatively']
negativewords_dict={}
for i in negativewords:
    negativewords_dict[i]=i

In [292]:
pwd

'/root/hald_autoupdate/2022_12_09'

for i,j in relation_dict_copy.items():
    for k in j:
        if k.get('openie'):
            del k['openie']

In [ ]:
import datetime
import networkx as nx
import matplotlib.pyplot as plt
from pylab import mpl
import re
from multiprocessing import Process, Manager, Pool
import os, time, random
def work(i,j,triple,abnormal, negative_sentences,networkx_raw):
    adv=sentence2adv.get(i)
    words=sentence2words.get(i)

    sentence=i
    # length=len(sentence2words.get(i))

    relations=sentence2relations.get(i)
    relations_display=[relation[-2] for relation in relations]
    verblist=sentence2verblist.get(i)
    verbprototype=sentence2verbprototype.get(i)
    verbprototype_dict={}
    for verbp in verbprototype:
        verbprototype_dict.update(verbp)
    # for k in range(length):
    G = nx.Graph()  # 建立无向图G

    # 添加节点
    for word in words:
        G.add_node(word)

    G.add_node('root')

    # 添加边
    for r in range(len(relations)):
        if relations[r][4]=='root':
            G.add_edge(relations[r][0], 'root',labels=relations[r][4])
        else:
            G.add_edge(relations[r][0], relations[r][1],labels=relations[r][4])

    for k in j:
        if verblist:
            k1=k.get('target1').translate(str.maketrans({"-":  r"\-","\\": r"\\", "^":  r"\^","$":  r"\$","*":  r"\*",".":  r"\.","(":  r"\(",")":  r"\)","+":  r"\+","[":  r"\[","]":  r"\]",
                                      "{":  r"\{","}":  r"\}","|":  r"\|","?":  r"\?"}))
            k2=k.get('target2').translate(str.maketrans({"-":  r"\-","\\": r"\\", "^":  r"\^","$":  r"\$","*":  r"\*",".":  r"\.","(":  r"\(",")":  r"\)","+":  r"\+","[":  r"\[","]":  r"\]",
                                      "{":  r"\{","}":  r"\}","|":  r"\|","?":  r"\?"}))
            if re.search(k1,sentence) and re.search(k2,sentence):
                worddict={}
                for word in words:
                    worddict[word]=word
                if re.search(k1,sentence).span()[0] < re.search(k2,sentence).span()[0]:
                    k1_word=k.get('target1')
                    k2_word=k.get('target2')
                    if not worddict.get(k1_word):
                        for word in words:
                            if re.search(r'-',word) and re.search(k1,word):
                                k1_word=word
                    if not worddict.get(k2_word):
                        for word in words:
                            if re.search(r'-',word) and re.search(k2,word):
                                k2_word=word
                    try:
                        path=nx.shortest_path(G, source=re.split('[ -]',k1_word)[-1], target=re.split('[ -]',k2_word)[-1])
                    except:
                        abnormal.append(i)
                    else:
                        verbdict={}
                        for verb in verblist:
                            verbdict.update(verb)
                        verbrelation=''
                        for one in path:
                            if verbdict.get(one):
                                verbrelation=one

                        # verbprototype[verblist.index(verb)]    
                        if verbrelation :
                            negative=False
                            for index_passive,passive in enumerate(relations_display):
                                if passive=='aux:pass':
                                    if relations[index_passive][1]==verbrelation:
                                        verb_prototype=verbrelation
                                        break
                                elif passive=='acl' and relations[index_passive][-1]=='VBN':
                                    if relations[index_passive][0]==verbrelation:
                                        verb_prototype=verbrelation
                                        break
                            else:
                                for v in verbprototype:
                                    if v.get(verbrelation):
                                        verb_prototype=v.get(verbrelation)
                                        break
                            if verb_prototype:
                                for relation in relations:
                                    if relation[1]==verbrelation and negativewords_dict.get(relation[0]):
                                        negative=True
                                if not negative:
                                    triple.append({'triple':[k.get('target1'),verb_prototype,k.get('target2')],\
                                                        'PMID':k.get('PMID'), 'sentence':sentence,'AB':k.get('AB'),
                                                   'former_target':k.get('target1'),'former_entity':k.get('entity1'),'former_entity_type':k.get('target1_type'),
                                                   'verb':verbrelation,'verbprototype':verbprototype_dict.get(verbrelation),'latter_target':k.get('target2'),
                                                   'latter_entity':k.get('entity2'),'latter_entity_type':k.get('target2_type')})
                                    k['networkx']=[k.get('target1'),verb_prototype,k.get('target2')]
                                else:
                                    negative_sentences.append(i)
                else:
                    k1_word=k.get('target1')
                    k2_word=k.get('target2')
                    if not worddict.get(k1_word):
                        for word in words:
                            if re.search(r'-',word) and re.search(k1,word):
                                k1_word=word
                    if not worddict.get(k2_word):
                        for word in words:
                            if re.search(r'-',word) and re.search(k2,word):
                                k2_word=word
                    try:
                        path=nx.shortest_path(G, source=re.split('[ -]',k2_word)[-1], target=re.split('[ -]',k1_word)[-1])
                    except:
                        abnormal.append(i)
                    else:
                        verbdict={}
                        for verb in verblist:
                            verbdict.update(verb)
                        verbrelation=''
                        for one in path:
                            if verbdict.get(one):
                                verbrelation=one
                        # verbprototype[verblist.index(verb)]    
                        if verbrelation:
                            negative=False
                            for index_passive,passive in enumerate(relations_display):
                                if passive=='aux:pass':
                                    if relations[index_passive][1]==verbrelation:
                                        verb_prototype=verbrelation
                                        break
                                elif passive=='acl' and relations[index_passive][-1]=='VBN':
                                    if relations[index_passive][0]==verbrelation:
                                        verb_prototype=verbrelation
                                        break
                            else:
                                for v in verbprototype:
                                    if v.get(verbrelation):
                                        verb_prototype=v.get(verbrelation)
                                        break
                            if verb_prototype:
                                for relation in relations:
                                    if relation[1]==verbrelation and negativewords_dict.get(relation[0]):
                                        negative=True
                                if not negative:
                                    triple.append({'triple':[k.get('target2'),verb_prototype,k.get('target1')],\
                                                        'PMID':k.get('PMID'), 'sentence':sentence,'AB':k.get('AB'),
                                                   'former_target':k.get('target2'),'former_entity':k.get('entity2'),'former_entity_type':k.get('target2_type'),
                                                   'verb':verbrelation,'verbprototype':verbprototype_dict.get(verbrelation),'latter_target':k.get('target1'),
                                                   'latter_entity':k.get('entity1'),'latter_entity_type':k.get('target1_type')})
                                    k['networkx']=[k.get('target2'),verb_prototype,k.get('target1')]
                                else:
                                    negative_sentences.append(i)
        networkx_raw.append(k)
if __name__=='__main__':
    start=datetime.datetime.now()
    print('Parent process %s.' % os.getpid())
    p=Pool(40)
    manager=Manager()
    triple=manager.list()
    abnormal=manager.list()
    negative_sentences=manager.list()
    networkx_raw=manager.list()
    for i,j in relation_dict_copy.items():
        p.apply_async(work, args=(i,j,triple,abnormal,negative_sentences,networkx_raw))

    print('Waiting for all subprocesses done...')
    
    p.close()
    p.join()
    print('All subprocesses done.')

    end=datetime.datetime.now()
    print("The running time is "+str((end-start).seconds)+"s")  

In [ ]:
triple_list=list(triple)
abnormal_list=list(abnormal)
negative_list=list(negative_sentences)
networkx_raw_list=list(networkx_raw)
networkx_raw_dict={}
for i in networkx_raw_list:
    networkx_raw_dict[i.get('sentence')]=networkx_raw_dict.get(i.get('sentence'),[])+[i]

In [ ]:
raw_pick_pubtator=pickle.load(open('raw_pick_pubtator.pkl','rb'))
rawpickwithAB={}
for i in raw_pick_pubtator:
    rawpickwithAB[i.get('PMID')]=i

In [ ]:
networkx_raw_list[0]

In [ ]:
networkx_raw_dict={}
for i in networkx_raw_list:
    networkx_raw_dict[i.get('sentence')]=networkx_raw_dict.get(i.get('sentence'),[])+[i]

In [ ]:
len(networkx_raw_dict)

In [ ]:
be=['be','am','is','are','was','were','have been','has been']
bedict={}
for i in be:
    bedict[i]=i
can=['can','could','will','would','may','might']
candict={}
for i in can:
    candict[i]=i

In [ ]:
for i in networkx_raw_list:
    openie_mid=[]
    openie_final=[]
    if i.get('openie'):
        for j in i.get('openie'):
            if not bedict.get(j[1]) and not candict.get(j[1]):
                openie_mid.append(j)
        adv=sentence2adv.get(i.get('sentence'))
        adv_dict={}
        for l in adv:
            adv_dict.update(l)
        for k in openie_mid:
            for m in k[1].split(' '):
                if adv_dict.get(m):
                    break
            else:
                if k not in openie_final:
                    openie_final.append(k)
    i['openie_final']=openie_final
        

In [ ]:
for i in networkx_raw_list:
    allennlp_final=[]
    if i.get('allennlp'):
        for j in i.get('allennlp'):
            if not bedict.get(j[1]) and not candict.get(j[1]):
                allennlp_final.append(j)
    i['allennlp_final']=allennlp_final

In [ ]:
for i in networkx_raw_list:
    if i.get('networkx'):
        if bedict.get(i.get('networkx')[1]):
            i['networkx']=''
        elif i.get('networkx')[1]=='"':
            i['networkx']=''
    else:
        i['networkx']=''

In [ ]:
networkx_raw_final=[]
for i in networkx_raw_list:
    if i.get('networkx') or i.get('openie_final') or i.get('allennlp_final'):
        networkx_raw_final.append(i)

In [156]:
len(networkx_raw_final)

NameError: name 'networkx_raw_final' is not defined

In [ ]:
for i in networkx_raw_final:
    bidirection=False
    if i.get('networkx'):
        if i.get('networkx')[1]=='associated' or i.get('networkx')[1]=='associate' :
            i['bidirection']=True
            continue
    if i.get('allennlp'):
        for j in i.get('openie_final'):
            if j=='associated' or j=='associate' :
                bidirection=True
                break
    if i.get('openie_final'):
        for j in i.get('openie_final'):
            for k in j[1].split(' '):
                if k=='associated' or  k=='associate' :
                    bidirection=True
                    break
            else:
                continue
            break
    i['bidirection']=bidirection
    

In [46]:
import copy

In [ ]:
Relation_raw=copy.deepcopy(networkx_raw_final)

In [ ]:
pickle.dump(networkx_raw_final,open('networkx_raw_final.pkl','wb'))

In [22]:
networkx_raw_final=pickle.load(open('networkx_raw_final.pkl','rb'))

In [23]:
len(networkx_raw_final)

3801

In [24]:
len(networkx_raw_final)

3801

In [25]:
for i in networkx_raw_final:
    if i.get('allennlp_final') and len(i.get('allennlp_final'))>1:
        print(i)
        break

{'PMID': '35955698', 'entity1': 'LRG1', 'entity2': 'LEP', 'sentence': 'In this study, we investigated the association of LRG1 with obesity markers, including leptin and other adipokines in adolescents (11-14 years; n = 425).', 'AB': 'Leucine-rich alpha-2 glycoprotein1 (LRG1) is a member of the leucine-rich repeat (LRR) family that is implicated in multiple diseases, including cancer, aging, and heart failure, as well as diabetes and obesity. LRG1 plays a key role in diet-induced hepatosteatosis and insulin resistance by mediating the crosstalk between adipocytes and hepatocytes. LRG1 also promotes hepatosteatosis by upregulating de novo lipogenesis in the liver and suppressing fatty acid beta-oxidation. In this study, we investigated the association of LRG1 with obesity markers, including leptin and other adipokines in adolescents (11-14 years; n = 425). BMI-for-age classification based on WHO growth charts was used to define obesity. Plasma LRG1 was measured by ELISA, while other mark

In [26]:
for i in networkx_raw_final:
    if i.get('entity1')=='Disease' or i.get('entity2')=='Disease':
        print(i)

{'PMID': '36261438', 'entity1': 'Nervous System Diseases', 'entity2': 'Disease', 'sentence': 'We show that combined application of TNF and HBEGF, factors associated with neuropathologies, is sufficient to induce photoreceptor degeneration, glial pathologies, dyslamination, and scar formation: These develop simultaneously and progressively as one complex phenotype.', 'AB': 'Human organoids could facilitate research of complex and currently incurable neuropathologies, such as age-related macular degeneration (AMD) which causes blindness. Here, we establish a human retinal organoid system reproducing several parameters of the human retina, including some within the macula, to model a complex combination of photoreceptor and glial pathologies. We show that combined application of TNF and HBEGF, factors associated with neuropathologies, is sufficient to induce photoreceptor degeneration, glial pathologies, dyslamination, and scar formation: These develop simultaneously and progressively as 

In [27]:
shenmeqingkuang=[]
for i in networkx_raw_final:
    if i.get('entity1')=='Ceramides' and i.get('target1_type')=='Carbohydrate':
        shenmeqingkuang.append(i)
    elif i.get('entity2')=='Ceramides' and i.get('target2_type')=='Carbohydrate':
        shenmeqingkuang.append(i)

In [28]:
shenmeqingkuang

[{'PMID': '35302400',
  'entity1': 'Ceramides',
  'entity2': 'HIV Infections',
  'sentence': 'Concentrations of individual lysophosphatidylcholine (LPC) and ceramide (CER) species also differed by HIV serostatus (unadj-p<=0.05).',
  'AB': 'BACKGROUND: Age-related comorbidities and physical function impairments in aging people with HIV (PWH) can be improved through exercise interventions. The mechanisms underlying these improvements, including lipidomic changes, are unknown. METHODS: Sedentary adults (50-75 years old) with or without HIV participated in supervised endurance/resistance exercise for 24 weeks. Plasma lipid concentrations (~1200 lipid species from 13 lipid classes) at baseline and week 24 were measured by mass spectrometry. Given multiple comparisons, unadjusted and Benjamini-Hochberg corrected p-values are reported. Analyses are considered exploratory. RESULTS: Twenty-five PWH and 24 controls had paired samples at baseline and week 24. The change in total triacylglycerol (

In [29]:
len(shenmeqingkuang)

2

In [30]:
networkx_raw_final[0]

{'PMID': '35926182',
 'entity1': 'SF3B1',
 'entity2': 'EIF2AK1',
 'sentence': 'Here, using single-cell technologies and functional validation studies of primary SF3B1-mutant MDS-RS samples, we show that SF3B1 mutations lead to the activation of the EIF2AK1 pathway in response to heme deficiency and that targeting this pathway rescues aberrant erythroid differentiation and enables the red blood cell maturation of MDS-RS erythroblasts.',
 'AB': 'SF3B1 mutations, which occur in 20% of patients with myelodysplastic syndromes (MDS), are the hallmarks of a specific MDS subtype, MDS with ringed sideroblasts (MDS-RS), which is characterized by the accumulation of erythroid precursors in the bone marrow and primarily affects the elderly population. Here, using single-cell technologies and functional validation studies of primary SF3B1-mutant MDS-RS samples, we show that SF3B1 mutations lead to the activation of the EIF2AK1 pathway in response to heme deficiency and that targeting this pathway r

In [31]:
collections_Relation_raw=db.Relation_raw

In [ ]:
for i in Relation_raw:
    i['addtime']=now_time

In [ ]:
collections_Relation_raw.insert_many(Relation_raw)

In [175]:
collections_Relation_raw=db.Relation_raw

In [176]:
Relation_raw_all=list(collections_Relation_raw.find())

In [177]:
# 153050
len(Relation_raw_all)

153050

In [178]:
pwd

'/root/hald_autoupdate/2022_12_09'

for i in Relation_raw_all:
    if i.get('entity1')=='Ceramides' and i.get('target1_type')=='Carbohydrate':
        print(i)
        break
    elif i.get('entity2')=='Ceramides' and i.get('target2_type')=='Carbohydrate':
        print(i)
        break

In [179]:
Relation_raw_all[20]

{'_id': ObjectId('63a9bf280032074d8b2142c6'),
 'PMID': '36191742',
 'entity1': 'CR1',
 'entity2': 'Dementia',
 'sentence': 'As expected, a subset of dementia-associated SNVs were associated with ADNC risk in Europeans-e.g., BIN1, PICALM, CR1, MME, and COX7C.',
 'AB': 'The classic pathologic hallmarks of Alzheimer\'s disease (AD) are amyloid plaques and neurofibrillary tangles (AD neuropathologic changes, or ADNC). However, brains from individuals clinically diagnosed with "AD-type" (amnestic) dementia usually harbor heterogeneous neuropathologies in addition to, or other than, ADNC. We hypothesized that some AD-type dementia associated genetic single nucleotide variants (SNVs) identified from large genomewide association studies (GWAS) were associated with non-ADNC neuropathologies. To test this hypothesis, we analyzed data from multiple studies with available genotype and neuropathologic phenotype information. Clinical AD/dementia risk alleles of interest were derived from the very la

hellos=0
hellos2=0
for i in Relation_raw_all:
    if i.get('openie_final'):
        if i.get('networkx'):
            for j in i.get('openie_final'):
                if i.get('networkx')[1] in j[1].split(' '):
                    i['networkx']=j
                    hellos+=1
                    break
        if i.get('allennlp_final'):
            for k in range(len(i.get('allennlp_final'))):
                for l in i.get('openie_final'):
                    if i.get('allennlp_final')[k][1] in l[1].split(' '):
                        i.get('allennlp_final')[k][1]=l
                        hellos2+=1
                        break

In [180]:
for i in Relation_raw_all:
    del i['_id']

#1611
hellos

#427
hellos2

json.dump(Relation_raw_all,open('relationinfo.json','w'))

In [181]:
len(Relation_raw_all)

153050

In [182]:
Relation_raw_all_with_disease=[]
for i in Relation_raw_all:
    if i.get('target1_type')=='Disease':
        if Disease_dropout_entities.get(i.get('entity1')):
            continue
    if i.get('target2_type')=='Disease':
        if Disease_dropout_entities.get(i.get('entity2')):
            continue
    if i.get('target1_type')=='Mutation':
        if Mutation_dropout_entities.get(i.get('entity1')):
            continue
    if i.get('target2_type')=='Mutation':
        if Mutation_dropout_entities.get(i.get('entity2')):
            continue
    Relation_raw_all_with_disease.append(i)

In [189]:
all_es=json.load(open('results/all_es_lack.json'))

In [195]:
# multi_cheicals=[]
multi_chemical_type2={}
for i in all_es:
    if not multi_chemical_type2.get(i.get('entity')):
        multi_chemical_type2[i.get('entity')]=[i.get('type')]
    else:
        if i.get('type') not in multi_chemical_type2.get(i.get('entity')):
            multi_chemical_type2[i.get('entity')]+=[i.get('type')]

In [196]:
cxz=0
for i,j in multi_chemical_type2.items():
    if len(j)>1:
        cxz+=1
        print({i:j})
print(cxz)

{'Lipopolysaccharides': ['Carbohydrate', 'Lipid', 'Toxin']}
{'Vancomycin': ['Carbohydrate', 'Peptide']}
{'Ceramides': ['Carbohydrate', 'Lipid']}
{'Sphingomyelins': ['Carbohydrate', 'Lipid']}
{'Bleomycin': ['Carbohydrate', 'Peptide']}
{'Teicoplanin': ['Carbohydrate', 'Peptide']}
{'Lipid A': ['Carbohydrate', 'Lipid', 'Toxin']}
{'G(M1) Ganglioside': ['Carbohydrate', 'Lipid']}
{'Gangliosides': ['Carbohydrate', 'Lipid']}
{'Glycosphingolipids': ['Carbohydrate', 'Lipid']}
{'Sulfoglycosphingolipids': ['Carbohydrate', 'Lipid']}
{'Glycolipids': ['Carbohydrate', 'Lipid']}
{'G(M3) Ganglioside': ['Carbohydrate', 'Lipid']}
{'Glycopeptides': ['Carbohydrate', 'Peptide']}
{'Glycosylphosphatidylinositols': ['Carbohydrate', 'Lipid']}
{'Globosides': ['Carbohydrate', 'Lipid']}
{'Ristocetin': ['Carbohydrate', 'Peptide']}
{'Glucosylceramides': ['Carbohydrate', 'Lipid']}
{'Holothurin': ['Carbohydrate', 'Toxin']}
{'Glycerylphosphorylcholine': ['Carbohydrate', 'Lipid']}
{'Glycerophosphates': ['Carbohydrate', 'L

multi_chemical_type1=json.load(open('multi_chemical_type.json'))
multi_chemical_type2=json.load(open('/root/hald/step6_entities_info/chemical/multi_chemical_type.json'))

multi_chemical_type1

multi_chemical_type2

len(multi_chemical_type2)

for i,j in multi_chemical_type1.items():
    if not multi_chemical_type2.get(i):
        multi_chemical_type2[i]=j

len(multi_chemical_type2)

In [187]:
len(Relation_raw_all_with_disease)

121897

In [188]:
Relation_raw_all_with_disease[0]

{'PMID': '35926182',
 'entity1': 'SF3B1',
 'entity2': 'EIF2AK1',
 'sentence': 'Here, using single-cell technologies and functional validation studies of primary SF3B1-mutant MDS-RS samples, we show that SF3B1 mutations lead to the activation of the EIF2AK1 pathway in response to heme deficiency and that targeting this pathway rescues aberrant erythroid differentiation and enables the red blood cell maturation of MDS-RS erythroblasts.',
 'AB': 'SF3B1 mutations, which occur in 20% of patients with myelodysplastic syndromes (MDS), are the hallmarks of a specific MDS subtype, MDS with ringed sideroblasts (MDS-RS), which is characterized by the accumulation of erythroid precursors in the bone marrow and primarily affects the elderly population. Here, using single-cell technologies and functional validation studies of primary SF3B1-mutant MDS-RS samples, we show that SF3B1 mutations lead to the activation of the EIF2AK1 pathway in response to heme deficiency and that targeting this pathway r

In [197]:
import copy
Relation_raw_all_final=[]
for i in Relation_raw_all_with_disease:
    if multi_chemical_type2.get(i.get('entity1')) and multi_chemical_type2.get(i.get('entity2')):
        for j in multi_chemical_type2.get(i.get('entity1')):
            for k in multi_chemical_type2.get(i.get('entity2')):
                mid=copy.deepcopy(i)
                mid.update({'target1_type':j,'target2_type':k})
                Relation_raw_all_final.append(mid)
    else:
        Relation_raw_all_final.append(i)
        if multi_chemical_type2.get(i.get('entity1')):
            for j in multi_chemical_type2.get(i.get('entity1')):
                if i.get('target1_type') != j:
                    mid=copy.deepcopy(i)
                    mid.update({'target1_type':j})
                    Relation_raw_all_final.append(mid)
        elif multi_chemical_type2.get(i.get('entity2')):
            for j in multi_chemical_type2.get(i.get('entity2')):
                if i.get('target2_type') != j:
                    mid=copy.deepcopy(i)
                    mid.update({'target2_type':j})
                    Relation_raw_all_final.append(mid)

In [198]:
#122534
len(Relation_raw_all_final)

122534

direction=set()
direction_list=[]
direction_type=[]
for i in Relation_raw_all_final:
    if i.get('networkx'):
        if i.get('openie_final'):
            if i.get('networkx')[0]!= i.get('target1'):
                print(i)
                break

In [199]:
direction=set()
direction_list=[]
direction_type=[]
for i in Relation_raw_all_final:

    if i.get('openie_final'):
        if i.get('openie_final')[0][0]!=i.get('target1'):
            i['entity2'],i['entity1']=i['entity1'],i['entity2']
            i['target2_type'],i['target1_type']=i['target1_type'],i['target2_type']
            i['target2'],i['target1']=i['target1'],i['target2']
    elif i.get('networkx'):
        if i.get('networkx')[0]!= i.get('target1'):
            i['entity2'],i['entity1']=i['entity1'],i['entity2']
            i['target2_type'],i['target1_type']=i['target1_type'],i['target2_type']
            i['target2'],i['target1']=i['target1'],i['target2']
    elif i.get('allennlp_final'):
        if i.get('allennlp_final')[0][0]!=i.get('target1'):
            i['entity2'],i['entity1']=i['entity1'],i['entity2']
            i['target2_type'],i['target1_type']=i['target1_type'],i['target2_type']
            i['target2'],i['target1']=i['target1'],i['target2']
    if i.get('bidirection'):
        if (i.get('entity2'),(i.get('entity1'))) not in direction :
            direction.add((i.get('entity2'),(i.get('entity1'))))
            direction_list.append((i.get('entity2'),(i.get('entity1'))))
            direction_type.append((i.get('target2_type'),(i.get('target1_type'))))
        if (i.get('entity1'),(i.get('entity2'))) not in direction:
            direction.add((i.get('entity1'),(i.get('entity2'))))
            direction_list.append((i.get('entity1'),(i.get('entity2'))))
            direction_type.append((i.get('target1_type'),(i.get('target2_type'))))
    else:
        direction.add((i.get('entity1'),(i.get('entity2'))))
        direction_list.append((i.get('entity1'),(i.get('entity2'))))
        direction_type.append((i.get('target1_type'),(i.get('target2_type'))))

In [200]:
Relation_raw_all_final[10]

{'PMID': '35556207',
 'entity1': 'rs1001179',
 'entity2': 'Cataract',
 'sentence': 'Association of SOD1 50 bp Indel, GPX1 (rs1800668), and CAT (rs1001179) genotypes with all clinical variables was investigated in cataract groups by regression statistics in SPSS  16.0.',
 'AB': 'Hyperglycemia plays a pronounced role in accelerating the process of aging due to high oxidative stress which triggers dyslipidemia and subsequently led to the progression of cataract. The aim of this study was to investigate lipid profile and its relationship with genotypes of SOD1, GPX1, and CAT variants in cataract patients. Total n = 680 samples were screened in four groups: senile cataract (SC), diabetic cataract (DC), type 2 diabetes mellitus (DM), and controls (CL). Lipid profile was estimated and compared between groups, and its correlation was tested with glycemic markers. Association of SOD1 50 bp Indel, GPX1 (rs1800668), and CAT (rs1001179) genotypes with all clinical variables was investigated in cat

In [201]:
pairs

['Gene',
 'RNA',
 'Carbohydrate',
 'Lipid',
 'Peptide',
 'Pharmaceutical_Preparations',
 'Protein',
 'Toxin',
 'Mutation',
 'Disease']

In [171]:
rules_pairs=['sentence2adv','sentence2relations','sentence2verblist','sentence2verbprototype','sentence2words']
names=locals()
for i in rules_pairs:
    names[i]={}
    for k in ['rules/files/'+i+str(j)+'.pkl' for j in range(1,22)]:
        names[i].update(pickle.load(open(k,'rb')))
    names[i].update(pickle.load(open('rules/files/'+i+'_old.pkl','rb')))

In [172]:
import copy
Relation_raw_all_final_copy=copy.deepcopy(Relation_raw_all_final)

In [173]:
be=['be','am','is','are','was','were','have been','has been']
bedict={}
for i in be:
    bedict[i]=i
can=['can','could','will','would','may','might']
candict={}
for i in can:
    candict[i]=i
countt=0
counttt=0
for i in Relation_raw_all_final_copy:
    verblist=sentence2verblist.get(i.get('sentence'))
    verbrelation=''
    real_verb=''
    verbdict={}
    for verb in verblist:
        verbdict.update(verb)
    for indexx,j in enumerate(i.get('allennlp_final')):
        one=j[1]
        if verbdict.get(one):
            verbrelation=one
            relations=sentence2relations.get(i.get('sentence'))
            relations_display=[relation[-2] for relation in relations]
            verbprototype=sentence2verbprototype.get(i.get('sentence'))
            verbprototype_dict={}
            for verbp in verbprototype:
                verbprototype_dict.update(verbp)
            for index_passive,passive in enumerate(relations_display):
                if passive=='aux:pass':
                    if relations[index_passive][1]==verbrelation:
                        verb_prototype=verbrelation

                        break
                elif passive=='acl' and relations[index_passive][-1]=='VBN':
                    if relations[index_passive][0]==verbrelation:
                        verb_prototype=verbrelation

                        break
            else:
                for v in verbprototype:
                    if v.get(verbrelation):
                        verb_prototype=v.get(verbrelation)
                        break
            if verb_prototype!=one:
                if countt<10:
                    countt+=1
                    # print(i.get('allennlp_final'))
                    # print(i.get('sentence'))
                    # print(verb_prototype)
            i['allennlp_final'][indexx][1]=verb_prototype
        elif bedict.get(one):
            i['allennlp_final'][indexx][1]=bedict.get(one)



sentence2relations.get('Then, a GC signature for ovarian aging was established based on six oxidative stress-related differentially expressed genes (MAPK1, STK24, AREG, ATG7, ANXA1, and PON2).')

Relation_raw_all_final[10]

for i in Relation_raw_all_final:
    if i.get('networkx') and i.get('networkx')[1]=='base':
        print(i)
        break

In [174]:
for i in Relation_raw_all_final_copy:
    if i.get('_id'):
        del i['_id']
json.dump(Relation_raw_all_final_copy,open('triplelist.json','w'))

In [90]:
len(direction)

48709

In [91]:
len(direction)

48709

In [92]:
len(direction_type)

48709

In [93]:
direction_type[:20]

[('DNA', 'DNA'),
 ('DNA', 'Disease'),
 ('DNA', 'DNA'),
 ('DNA', 'Disease'),
 ('DNA', 'DNA'),
 ('DNA', 'Disease'),
 ('DNA', 'Disease'),
 ('Mutation', 'Disease'),
 ('Disease', 'Disease'),
 ('Mutation', 'Disease'),
 ('DNA', 'Disease'),
 ('DNA', 'Disease'),
 ('DNA', 'DNA'),
 ('Lipid', 'DNA'),
 ('DNA', 'DNA'),
 ('DNA', 'Disease'),
 ('DNA', 'Disease'),
 ('Disease', 'DNA'),
 ('Disease', 'DNA'),
 ('Disease', 'DNA')]

In [94]:
direction_list[:20]

[('SF3B1', 'EIF2AK1'),
 ('SF3B1', 'Protoporphyria, Erythropoietic'),
 ('SOD1', 'GPX1'),
 ('GPX1', 'Cataract'),
 ('CAT', 'GPX1'),
 ('SOD1', 'Cataract'),
 ('CAT', 'Cataract'),
 ('rs1800668', 'Cataract'),
 ('Inflammation', 'Seizures'),
 ('rs1001179', 'Cataract'),
 ('CDKN1A', 'Seizures'),
 ('EGR1', 'Seizures'),
 ('CPT1A', 'TP53'),
 ('Fatty Acids', 'CPT1A'),
 ('ACAN', 'COL11A1'),
 ('WNT3', 'Sclerosis'),
 ('TNIP1', 'Sclerosis'),
 ('Dementia', 'BIN1'),
 ('Dementia', 'CR1'),
 ('Dementia', 'COX7C')]

In [141]:
filtered_direction_list=[]
filtered_direction_type=[]
for i in range(len(direction_list)):
    if direction_type[i][0]!='Mutation' and direction_type[i][1]!='Mutation' and direction_type[i][0]!='Disease' and direction_type[i][1]!='Disease':
        filtered_direction_list.append((direction_list[i][0],direction_list[i][1]))
        filtered_direction_type.append((direction_type[i][0],direction_type[i][1]))

In [142]:
len(filtered_direction_list)

4051

In [22]:
import pandas as pd

In [23]:
file=pd.read_table('../intact/intact.txt')

In [24]:
file.head()

,#ID(s) interactor A,ID(s) interactor B,Alt. ID(s) interactor A,Alt. ID(s) interactor B,Alias(es) interactor A,Alias(es) interactor B,Interaction detection method(s),Publication 1st author(s),Publication Identifier(s),Taxid interactor A,...,Checksum(s) interactor A,Checksum(s) interactor B,Interaction Checksum(s),Negative,Feature(s) interactor A,Feature(s) interactor B,Stoichiometry(s) interactor A,Stoichiometry(s) interactor B,Identification method participant A,Identification method participant B
0,uniprotkb:P49418,uniprotkb:O43426,intact:EBI-7121510|uniprotkb:Q75MK5|uniprotkb:...,intact:EBI-2821539|uniprotkb:O43425|uniprotkb:...,psi-mi:amph_human(display_long)|uniprotkb:AMPH...,psi-mi:synj1_human(display_long)|uniprotkb:SYN...,"psi-mi:""MI:0084""(phage display)",Cestra et al. (1999),pubmed:10542231|mint:MINT-5211933,taxid:9606(human)|taxid:9606(Homo sapiens),...,rogid:vrgVrVoYr45cUe4X6L/zBAE1RtU9606,rogid:RA73eMbCn6F7MD0ItxF/V7QbjqM9606,intact-crc:F4234557A3B54840|rigid:n+UcEH4PPLkF...,False,binding-associated region:626-695(MINT-376295),binding-associated region:1063-1070(MINT-376288),-,-,"psi-mi:""MI:0078""(nucleotide sequence identific...","psi-mi:""MI:0078""(nucleotide sequence identific..."
1,uniprotkb:P49418,intact:EBI-7121639,intact:EBI-7121510|uniprotkb:Q75MK5|uniprotkb:...,intact:MINT-8094608,psi-mi:amph_human(display_long)|uniprotkb:AMPH...,psi-mi:vrparrvlw(display_short)|psi-mi:EBI-712...,"psi-mi:""MI:0084""(phage display)",Cestra et al. (1999),pubmed:10542231|mint:MINT-5211933,taxid:9606(human)|taxid:9606(Homo sapiens),...,rogid:vrgVrVoYr45cUe4X6L/zBAE1RtU9606,rogid:lPy6gBhpgvyGSYgOqeHbjcqBtMQ-2,intact-crc:880913D9000BF26E|rigid:iRObbQsaUeX0...,False,binding-associated region:626-695(MINT-8094602),-,-,-,"psi-mi:""MI:0078""(nucleotide sequence identific...","psi-mi:""MI:0083""(peptide synthesis)"
2,uniprotkb:P49418,intact:EBI-7121654,intact:EBI-7121510|uniprotkb:Q75MK5|uniprotkb:...,intact:MINT-8094645,psi-mi:amph_human(display_long)|uniprotkb:AMPH...,psi-mi:hrpvrraap(display_short)|psi-mi:EBI-712...,"psi-mi:""MI:0084""(phage display)",Cestra et al. (1999),pubmed:10542231|mint:MINT-5211933,taxid:9606(human)|taxid:9606(Homo sapiens),...,rogid:vrgVrVoYr45cUe4X6L/zBAE1RtU9606,rogid:GsDnsVmAo8ZZoKTZ4/uDb2x+h3c-2,intact-crc:B68245E1F0508124|rigid:SlinPP5vIczF...,False,binding-associated region:626-695(MINT-8094630),-,-,-,"psi-mi:""MI:0078""(nucleotide sequence identific...","psi-mi:""MI:0083""(peptide synthesis)"
3,uniprotkb:P49418,intact:EBI-7121715,intact:EBI-7121510|uniprotkb:Q75MK5|uniprotkb:...,intact:MINT-8094663,psi-mi:amph_human(display_long)|uniprotkb:AMPH...,psi-mi:vrptraada(display_short)|psi-mi:EBI-712...,"psi-mi:""MI:0084""(phage display)",Cestra et al. (1999),pubmed:10542231|mint:MINT-5211933,taxid:9606(human)|taxid:9606(Homo sapiens),...,rogid:vrgVrVoYr45cUe4X6L/zBAE1RtU9606,rogid:M+f6GE3SQXrMBiIVttW1MKhz3M0-2,intact-crc:8F7F46DF027DF34E|rigid:uiWbDe6gaj1X...,False,binding-associated region:626-695(MINT-8094654),-,-,-,"psi-mi:""MI:0078""(nucleotide sequence identific...","psi-mi:""MI:0083""(peptide synthesis)"
4,intact:EBI-7121765,uniprotkb:P49418,intact:MINT-8094691,intact:EBI-7121510|uniprotkb:Q75MK5|uniprotkb:...,psi-mi:xrprrghal(display_short)|psi-mi:EBI-712...,psi-mi:amph_human(display_long)|uniprotkb:AMPH...,"psi-mi:""MI:0084""(phage display)",Cestra et al. (1999),pubmed:10542231|mint:MINT-5211933,"taxid:-2(chemical synthesis)|taxid:-2(""Chemica...",...,rogid:cilZXZ3YLDI1s7n8WNRyUE6EnJQ-2,rogid:vrgVrVoYr45cUe4X6L/zBAE1RtU9606,intact-crc:2EF43A66D9B2D428|rigid:/ki1zA46M4Q+...,False,-,binding-associated region:626-695(MINT-8094680),-,-,"psi-mi:""MI:0083""(peptide synthesis)","psi-mi:""MI:0078""(nucleotide sequence identific..."


In [155]:
len(file)

1194447

In [188]:
interactorsA=[]
interactorsB=[]
for i in range(len(file)):
    interactorA={}
    interactorB={}
    if file.iloc[i,4]!='-' and file.iloc[i,5]!='-':
        for spl1 in file.iloc[i,4].split('|'):
            if re.search(r'(.*:)(.*)(\(.*\))',spl1):
                interactorA[re.search(r'(.*:)(.*)(\(.*\))',spl1).groups()[1].lower()]=file.iloc[i,9]
        for spl2 in file.iloc[i,5].split('|'):
            if re.search(r'(.*:)(.*)(\(.*\))',spl2):
                interactorB[re.search(r'(.*:)(.*)(\(.*\))',spl2).groups()[1].lower()]=file.iloc[i,10]
    interactorsA.append(interactorA)
    interactorsB.append(interactorB)

In [189]:
len(interactorsA)

1194447

In [190]:
def work(k,experiment,experiment_type, work_list):
    for i in range(len(interactorsA)):
        if interactorsA[i].get(filtered_direction_list[k][0].lower()) and interactorsB[i].get(filtered_direction_list[k][1].lower()):
        # if interactorsA[i].get(k[0].lower()):
            if filtered_direction_type[k][0] in ['DNA','RNA'] and filtered_direction_type[k][1] in ['DNA','RNA']:
                if re.search(r'Homo sapiens', interactorsA[i].get(filtered_direction_list[k][0].lower())) and re.search(r'Homo sapiens', interactorsB[i].get(filtered_direction_list[k][1].lower())):
                    experiment.append((filtered_direction_list[k][0],filtered_direction_list[k][1]))
                    experiment_type.append((filtered_direction_type[k][0],filtered_direction_type[k][1]))
                    work_list.append(i)
                    break
            elif re.search(r'Homo sapiens', interactorsA[i].get(filtered_direction_list[k][0].lower())):
                if re.search(r'Homo sapiens', interactorsA[i].get(filtered_direction_list[k][0].lower())):
                    experiment.append((filtered_direction_list[k][0],filtered_direction_list[k][1]))
                    experiment_type.append((filtered_direction_type[k][0],filtered_direction_type[k][1]))
                    work_list.append(i)
                    break
            elif re.search(r'Homo sapiens', interactorsB[i].get(filtered_direction_list[k][1].lower())):
                if re.search(r'Homo sapiens', interactorsB[i].get(filtered_direction_list[k][1].lower())):
                    experiment.append((filtered_direction_list[k][0],filtered_direction_list[k][1]))
                    experiment_type.append((filtered_direction_type[k][0],filtered_direction_type[k][1]))
                    work_list.append(i)
                    break
            else:
                experiment.append((filtered_direction_list[k][0],filtered_direction_list[k][1]))
                experiment_type.append((filtered_direction_type[k][0],filtered_direction_type[k][1]))
                work_list.append(i)
                break
        elif interactorsA[i].get(filtered_direction_list[k][1].lower()) and interactorsB[i].get(filtered_direction_list[k][0].lower()):
        # if interactorsA[i].get(k[0].lower()):
            if filtered_direction_type[k][1] in ['DNA','RNA'] and filtered_direction_type[k][0] in ['DNA','RNA']:
                if re.search(r'Homo sapiens', interactorsA[i].get(filtered_direction_list[k][1].lower())) and re.search(r'Homo sapiens', interactorsB[i].get(filtered_direction_list[k][0].lower())):
                    experiment.append((filtered_direction_list[k][1],filtered_direction_list[k][0]))
                    experiment_type.append((filtered_direction_type[k][1],filtered_direction_type[k][0]))
                    work_list.append(i)
                    break
            elif re.search(r'Homo sapiens', interactorsA[i].get(filtered_direction_list[k][1].lower())):
                if re.search(r'Homo sapiens', interactorsA[i].get(filtered_direction_list[k][1].lower())):
                    experiment.append((filtered_direction_list[k][1],filtered_direction_list[k][0]))
                    experiment_type.append((filtered_direction_type[k][1],filtered_direction_type[k][0]))
                    work_list.append(i)
                    break
            elif re.search(r'Homo sapiens', interactorsB[i].get(filtered_direction_list[k][0].lower())):
                if re.search(r'Homo sapiens', interactorsB[i].get(filtered_direction_list[k][1].lower())):
                    experiment.append((filtered_direction_list[k][1],filtered_direction_list[k][0]))
                    experiment_type.append((filtered_direction_type[k][1],filtered_direction_type[k][0]))
                    work_list.append(i)
                    break
            else:
                experiment.append((filtered_direction_list[k][1],filtered_direction_list[k][0]))
                experiment_type.append((filtered_direction_type[k][1],filtered_direction_type[k][0]))
                work_list.append(i)
                break
if __name__=='__main__':
    start=datetime.datetime.now()
    print('Parent process %s.' % os.getpid())
    p=Pool(60)
    manager=Manager()
    work_list=manager.list()
    experiment=manager.list()
    experiment_type=manager.list()
    for k in range(len(filtered_direction_list)):
        p.apply_async(work, args=(k,experiment,experiment_type,work_list))
    print('Waiting for all subprocesses done...')
    
    p.close()
    p.join()
    print('All subprocesses done.')

    end=datetime.datetime.now()
    print("The running time is "+str((end-start).seconds)+"s")  

Parent process 81816.
Waiting for all subprocesses done...
All subprocesses done.
The running time is 131s


In [191]:
# 374
len(list(work_list))

322

In [192]:
file.iloc[list(work_list)[-8],:]

#ID(s) interactor A                                                                                                                                                                                                           uniprotkb:P28340
ID(s) interactor B                                                                                                                                                                                                            uniprotkb:Q15054
Alt. ID(s) interactor A                                  intact:EBI-716569|ensembl:ENSP00000406046.1|ensembl:ENSP00000472600.2|ensembl:ENSP00000472607.2|ensembl:ENSP00000473052.1|ensembl:ENSP00000510052.1|uniprotkb:Q8NER3|uniprotkb:Q96H98
Alt. ID(s) interactor B                                                                                                                         intact:EBI-864956|uniprotkb:B7ZAI6|uniprotkb:Q32MZ9|uniprotkb:Q32N00|ensembl:ENSP00000263681.2
Alias(es) interactor A                 psi-m

In [193]:
file.iloc[list(work_list)[-8],23]

'refseq:NP_006582.1|dip:DIP-35772N|go:"GO:0006261"(DNA-dependent DNA replication)|go:"GO:0006271"(DNA strand elongation involved in DNA replication)|go:"GO:0006297"(nucleotide-excision repair, DNA gap filling)|go:"GO:0006298"(mismatch repair)|go:"GO:0016035"(zeta DNA polymerase complex)|go:"GO:0030674"(protein-macromolecule adaptor activity)|go:"GO:0042276"(error-prone translesion synthesis)|go:"GO:0043625"(delta DNA polymerase complex)|go:"GO:0071897"(DNA biosynthetic process)|go:"GO:1904161"(DNA synthesis involved in UV-damage excision repair)|interpro:IPR019038|interpro:IPR041913|mint:Q15054|rcsb pdb:1U76|rcsb pdb:2N1G|rcsb pdb:3E0J|rcsb pdb:6S1M|rcsb pdb:6S1N|rcsb pdb:6S1O|rcsb pdb:6TNY|rcsb pdb:6TNZ|reactome:R-HSA-110314|reactome:R-HSA-174411|reactome:R-HSA-174414|reactome:R-HSA-174417|reactome:R-HSA-174437|reactome:R-HSA-5358565|reactome:R-HSA-5358606|reactome:R-HSA-5651801|reactome:R-HSA-5656169|reactome:R-HSA-5685942|reactome:R-HSA-5696397|reactome:R-HSA-5696400|reactome:R-HSA-

In [194]:
list(experiment)[-8:-4]

[('POLD1', 'POLD3'), ('POLD1', 'POLD2'), ('CTSK', 'CTSV'), ('Glycogen', 'AGL')]

In [196]:
for i in list(experiment_type):
    if i[0]!= 'DNA' or i[1]!= 'DNA':
        print(i)

('Lipid', 'DNA')
('Lipid', 'DNA')
('Carbohydrate', 'DNA')
('Carbohydrate', 'DNA')
('Carbohydrate', 'DNA')
('Carbohydrate', 'DNA')


In [197]:
for i in range(len(list(experiment_type))):
    if list(experiment_type)[i][0]!= 'DNA' or list(experiment_type)[i][1]!= 'DNA':
        print(i)

87
88
101
317
318
319


In [200]:
for i in [87,88,101,317,318,319]:
    print(list(experiment)[i])

('Cholesterol', 'NPC2')
('Cholesterol', 'NPC1')
('Heparin', 'PF4')
('Glycogen', 'AGL')
('Heparin', 'APOE')
('Heparin', 'ANXA2')


In [199]:
file.iloc[list(work_list)[87],:]

#ID(s) interactor A                                                                                                                                                                                                        chebi:"CHEBI:16113"
ID(s) interactor B                                                                                                                                                                                                            uniprotkb:P61916
Alt. ID(s) interactor A                                                                                                                                                                                                     intact:EBI-2368700
Alt. ID(s) interactor B                                                                                                                        intact:EBI-2368946|uniprotkb:Q15668|uniprotkb:Q29413|uniprotkb:B4DQV7|ensembl:ENSP00000451112.2
Alias(es) interactor A                      

In [198]:
list(experiment)[87]

('Cholesterol', 'NPC2')

In [165]:
list(experiment_type)[350:]

[('DNA', 'DNA'),
 ('DNA', 'DNA'),
 ('DNA', 'DNA'),
 ('DNA', 'DNA'),
 ('DNA', 'DNA'),
 ('DNA', 'DNA'),
 ('DNA', 'DNA'),
 ('DNA', 'DNA'),
 ('DNA', 'DNA'),
 ('DNA', 'DNA'),
 ('DNA', 'DNA'),
 ('DNA', 'DNA'),
 ('DNA', 'DNA'),
 ('DNA', 'DNA'),
 ('DNA', 'DNA'),
 ('DNA', 'DNA'),
 ('Carbohydrate', 'DNA'),
 ('DNA', 'Carbohydrate'),
 ('DNA', 'Carbohydrate'),
 ('DNA', 'Carbohydrate'),
 ('DNA', 'DNA'),
 ('DNA', 'DNA'),
 ('DNA', 'DNA'),
 ('DNA', 'DNA')]

In [168]:
len(filtered_direction_type)

4051

In [173]:
Car=[]
for i in range(len(filtered_direction_type)):
    if filtered_direction_type[i][0]=='Carbohydrate' :
        Car.append(filtered_direction_list[i][0])
    elif filtered_direction_type[i][1]=='Carbohydrate':
        Car.append(filtered_direction_list[i][1])

In [175]:
Car[:50]

['Glucose',
 'Glucose',
 'Glucose',
 'Anthracyclines',
 'Anthracyclines',
 'Anthracyclines',
 'Glucose',
 'Glucose',
 'Glucose',
 'Glucose',
 'Glucose',
 'Glucose',
 'Doxorubicin',
 'Doxorubicin',
 'Doxorubicin',
 'Doxorubicin',
 'Glucose',
 'Glucose',
 'Glucose',
 'Glucose',
 'Sugars',
 'Sugars',
 'Doxorubicin',
 'Glucose',
 'Doxorubicin',
 'Fructose',
 'Fructose',
 'Heparitin Sulfate',
 'Heparitin Sulfate',
 'Glucose',
 'Glycogen',
 'Glucose',
 'Glycogen',
 'Glucose',
 'Glucose',
 'Glucose',
 'Ascorbic Acid',
 'Glucosamine',
 'Glucosamine',
 'Glucose',
 'Glycosaminoglycans',
 'Glucose',
 'Glucose',
 'Starch',
 'Anthracyclines',
 'Glucose',
 'Glucose',
 'Glucose',
 'Glucose',
 'Glucose']

In [121]:
lineindex=[]
for i in range(len(file)):
    if file.loc[i,'Taxid interactor A'].split('|')[0]=='taxid:9606(human)' and file.loc[i,'Taxid interactor B'].split('|')[0]=='taxid:9606(human)':
        lineindex.append(i)

In [122]:
# 632740
len(lineindex)

632740

In [123]:
df_intact=file.iloc[lineindex,:]

In [124]:
pd.set_option('display.max_columns', None) 
pd.set_option('display.max_rows', None) 
pd.set_option('display.expand_frame_repr', False) 
pd.set_option('display.max_colwidth', 200)

In [125]:
df_intact.head(1)

,#ID(s) interactor A,ID(s) interactor B,Alt. ID(s) interactor A,Alt. ID(s) interactor B,Alias(es) interactor A,Alias(es) interactor B,Interaction detection method(s),Publication 1st author(s),Publication Identifier(s),Taxid interactor A,Taxid interactor B,Interaction type(s),Source database(s),Interaction identifier(s),Confidence value(s),Expansion method(s),Biological role(s) interactor A,Biological role(s) interactor B,Experimental role(s) interactor A,Experimental role(s) interactor B,Type(s) interactor A,Type(s) interactor B,Xref(s) interactor A,Xref(s) interactor B,Interaction Xref(s),Annotation(s) interactor A,Annotation(s) interactor B,Interaction annotation(s),Host organism(s),Interaction parameter(s),Creation date,Update date,Checksum(s) interactor A,Checksum(s) interactor B,Interaction Checksum(s),Negative,Feature(s) interactor A,Feature(s) interactor B,Stoichiometry(s) interactor A,Stoichiometry(s) interactor B,Identification method participant A,Identification method participant B
0,uniprotkb:P49418,uniprotkb:O43426,intact:EBI-7121510|uniprotkb:Q75MK5|uniprotkb:Q75MM3|uniprotkb:A4D1X9|intact:MINT-109264|uniprotkb:O43538|uniprotkb:A4D1X8|uniprotkb:Q75MJ8|uniprotkb:Q8N4G0|ensembl:ENSP00000348602.2,intact:EBI-2821539|uniprotkb:O43425|uniprotkb:O94984|uniprotkb:Q4KMR1|ensembl:ENSP00000501426.1,psi-mi:amph_human(display_long)|uniprotkb:AMPH(gene name)|psi-mi:AMPH(display_short)|uniprotkb:AMPH1(gene name synonym),"psi-mi:synj1_human(display_long)|uniprotkb:SYNJ1(gene name)|psi-mi:SYNJ1(display_short)|uniprotkb:KIAA0910(gene name synonym)|uniprotkb:Synaptic inositol 1,4,5-trisphosphate 5-phosphatase 1(gene n...","psi-mi:""MI:0084""(phage display)",Cestra et al. (1999),pubmed:10542231|mint:MINT-5211933,taxid:9606(human)|taxid:9606(Homo sapiens),taxid:9606(human)|taxid:9606(Homo sapiens),"psi-mi:""MI:0407""(direct interaction)","psi-mi:""MI:0471""(MINT)",intact:EBI-7121552|mint:MINT-16056,intact-miscore:0.56,-,"psi-mi:""MI:0499""(unspecified role)","psi-mi:""MI:0499""(unspecified role)","psi-mi:""MI:0498""(prey)","psi-mi:""MI:0496""(bait)","psi-mi:""MI:0326""(protein)","psi-mi:""MI:0326""(protein)","refseq:NP_001626.1|refseq:NP_647477.1|ensembl:ENSG00000078053.17(gene)|ensembl:ENST00000356264.7(transcript)|go:""GO:0005543""(phospholipid binding)|go:""GO:0005829""(cytosol)|go:""GO:0005886""(plasma m...","refseq:NP_001153774.1|refseq:NP_001153778.1|refseq:NP_003886.3|refseq:NP_982271.2|refseq:XP_016883989.1|ensembl:ENSG00000159082.18(gene)|ensembl:ENST00000674308.1(transcript)|go:""GO:0003723""(RNA b...",-,function:May participate in mechanisms of regulated exocytosis in synapses and certain endocrine cell types. May control the properties of the membrane associated cytoskeleton|comment:mint|comment...,"comment:""Stoichiometry: 1.0""",comment:homomint|comment:domino|comment:mint,taxid:-1(in vitro)|taxid:-1(In vitro),-,2001/01/10,2014/10/16,rogid:vrgVrVoYr45cUe4X6L/zBAE1RtU9606,rogid:RA73eMbCn6F7MD0ItxF/V7QbjqM9606,intact-crc:F4234557A3B54840|rigid:n+UcEH4PPLkFnIyvBiXrLefK/xU,False,binding-associated region:626-695(MINT-376295),binding-associated region:1063-1070(MINT-376288),-,-,"psi-mi:""MI:0078""(nucleotide sequence identification)","psi-mi:""MI:0078""(nucleotide sequence identification)"


In [134]:
small_molecules_index=[]
for i in range(len(df_intact)):
    if re.search('(.*\()(.*)(\))',df_intact.iloc[i,20]).groups()[1]=='small molecule' or re.search('(.*\()(.*)(\))',df_intact.iloc[i,21].lower()).groups()[1]=='small molecule':
        small_molecules_index.append(i)


In [132]:
len(small_molecules_index)

0

In [127]:
df_small_molecules=df_intact.iloc[small_molecules_index,:]

In [128]:
df_small_molecules.head(5)

,#ID(s) interactor A,ID(s) interactor B,Alt. ID(s) interactor A,Alt. ID(s) interactor B,Alias(es) interactor A,Alias(es) interactor B,Interaction detection method(s),Publication 1st author(s),Publication Identifier(s),Taxid interactor A,Taxid interactor B,Interaction type(s),Source database(s),Interaction identifier(s),Confidence value(s),Expansion method(s),Biological role(s) interactor A,Biological role(s) interactor B,Experimental role(s) interactor A,Experimental role(s) interactor B,Type(s) interactor A,Type(s) interactor B,Xref(s) interactor A,Xref(s) interactor B,Interaction Xref(s),Annotation(s) interactor A,Annotation(s) interactor B,Interaction annotation(s),Host organism(s),Interaction parameter(s),Creation date,Update date,Checksum(s) interactor A,Checksum(s) interactor B,Interaction Checksum(s),Negative,Feature(s) interactor A,Feature(s) interactor B,Stoichiometry(s) interactor A,Stoichiometry(s) interactor B,Identification method participant A,Identification method participant B


In [130]:
df_intact.iloc[10,20]

'psi-mi:"MI:0326"(protein)'

In [ ]:
psi-mi:"MI:0328"(small molecule)

In [129]:
re.search('(.*\()(.*)(\))','psi-mi:"MI:0328"(small molecule)').groups()[1]

'small molecule'

In [118]:
for i in range(len(df_intact)):
    if re.search('apoe',df_intact.iloc[i,4].lower()):
        print(df_intact.iloc[i,:])
        break


#ID(s) interactor A                                                                                                                                                                                                           uniprotkb:P02649
ID(s) interactor B                                                                                                                                                                                                            uniprotkb:P00738
Alt. ID(s) interactor A                                                                                                                        intact:EBI-1222467|uniprotkb:C0JYY5|uniprotkb:Q9P2S4|uniprotkb:B2RC15|ensembl:ENSP00000252486.3
Alt. ID(s) interactor B                                   intact:EBI-1220767|uniprotkb:P00737|uniprotkb:Q0VAC4|uniprotkb:Q2PP15|uniprotkb:Q3B7J0|uniprotkb:Q6LBY9|uniprotkb:B0AZL5|uniprotkb:Q9UC67|uniprotkb:Q0VAC5|ensembl:ENSP00000348170.5
Alias(es) interactor A                      

In [ ]:
for i in range(len(df_intact)):
    for spl1 in df_intact.iloc[i,4].split('|'):
        interactorA[re.search(r'(.*:)(.*)(\(.*\))',spl1).groups()[1].lower()]=re.search(r'(.*:)(.*)(\(.*\))',spl1).groups()[1]
    for spl2 in df_intact.iloc[i,5].split('|'):
        interactorB[re.search(r'(.*:)(.*)(\(.*\))',spl2).groups()[1].lower()]=re.search(r'(.*:)(.*)(\(.*\))',spl2).groups()[1]
    interactorsA.append(interactorA)
    interactorsB.append(interactorB)

In [61]:
interactorsA=[]
interactorsB=[]
for i in range(len(df_intact)):
    interactorA={}
    interactorB={}
    for spl1 in df_intact.iloc[i,4].split('|'):
        interactorA[re.search(r'(.*:)(.*)(\(.*\))',spl1).groups()[1].lower()]=re.search(r'(.*:)(.*)(\(.*\))',spl1).groups()[1]
    for spl2 in df_intact.iloc[i,5].split('|'):
        interactorB[re.search(r'(.*:)(.*)(\(.*\))',spl2).groups()[1].lower()]=re.search(r'(.*:)(.*)(\(.*\))',spl2).groups()[1]
    interactorsA.append(interactorA)
    interactorsB.append(interactorB)

In [77]:
interactorsB[2]

{'synj1_human': 'synj1_human',
 'synj1': 'SYNJ1',
 'kiaa0910': 'KIAA0910',
 'synaptic inositol 1,4,5-trisphosphate 5-phosphatase 1': 'Synaptic inositol 1,4,5-trisphosphate 5-phosphatase 1'}

In [62]:
print(len(interactorsA))

632740


In [63]:
print(len(interactorsB))

632740


In [64]:
interactorsA[-1]

{'af1q_human': 'af1q_human', 'mllt11': 'MLLT11', 'af1q': 'AF1Q'}

In [111]:
def work(k,experiment,experiment_type):
    for i in range(len(interactorsA)):
        if (interactorsA[i].get(direction_list[k][0].lower()) and interactorsB[i].get(direction_list[k][1].lower())) or \
        (interactorsA[i].get(direction_list[k][1].lower()) and interactorsB[i].get(direction_list[k][0].lower())):
        # if interactorsA[i].get(k[0].lower()):
            experiment.append((direction_list[k][0],direction_list[k][1]))
            experiment_type.append((direction_type[k][0],direction_type[k][1]))
            break
if __name__=='__main__':
    start=datetime.datetime.now()
    print('Parent process %s.' % os.getpid())
    p=Pool(60)
    manager=Manager()
    experiment=manager.list()
    experiment_type=manager.list()
    for k in range(len(direction_list)):
        p.apply_async(work, args=(k,experiment,experiment_type))
    print('Waiting for all subprocesses done...')
    
    p.close()
    p.join()
    print('All subprocesses done.')

    end=datetime.datetime.now()
    print("The running time is "+str((end-start).seconds)+"s")  

Parent process 81816.
Waiting for all subprocesses done...
All subprocesses done.
The running time is 955s


In [110]:
direction_type[20][0]

'DNA'

In [112]:
print(len(list(experiment)))
print(len(list(experiment_type)))

316
316


In [113]:
experiment_final=list(experiment)

In [114]:
experiment_type_final=list(experiment_type)

In [115]:
for i in range(10):
    print(experiment_final[i])
    print(experiment_type_final[i])

('AR', 'KLK3')
('DNA', 'DNA')
('TP53', 'MDM2')
('DNA', 'DNA')
('TP53', 'CDKN1A')
('DNA', 'DNA')
('AKT1', 'MTOR')
('DNA', 'DNA')
('PINK1', 'RHOT1')
('DNA', 'DNA')
('RHOT1', 'PINK1')
('DNA', 'DNA')
('TP53', 'SIRT1')
('DNA', 'DNA')
('RHOT1', 'LRRK2')
('DNA', 'DNA')
('LRRK2', 'RHOT1')
('DNA', 'DNA')
('GAR1', 'NOP10')
('DNA', 'DNA')


In [116]:
typetype=[]
for i in experiment_type:
    if i[0]!='DNA':
        typetype.append(i[0])
    if i[1]!='DNA':
        typetype.append(i[1])

In [117]:
len(typetype)

0

In [ ]:
experiments=[]
for i in range(len(interactorsA)):
    if interactorsA[i].get('apoe') :
        experiments.append((interactorsA[i],interactorsB[i]))
        

In [84]:
csaca=[]
for k in direction_list:
    if k[0].lower()=='apoe':
        csaca.append(k[0])

In [85]:
len(csaca)

144

In [ ]:
test=[]
for k in direction_list:
    if 

In [79]:
for i in range(10):
    print(interactorsA[i])
    print(interactorsB[i])

{'amph_human': 'amph_human', 'amph': 'AMPH', 'amph1': 'AMPH1'}
{'synj1_human': 'synj1_human', 'synj1': 'SYNJ1', 'kiaa0910': 'KIAA0910', 'synaptic inositol 1,4,5-trisphosphate 5-phosphatase 1': 'Synaptic inositol 1,4,5-trisphosphate 5-phosphatase 1'}
{'amph_human': 'amph_human', 'amph': 'AMPH', 'amph1': 'AMPH1'}
{'synj1_human': 'synj1_human', 'synj1': 'SYNJ1', 'kiaa0910': 'KIAA0910', 'synaptic inositol 1,4,5-trisphosphate 5-phosphatase 1': 'Synaptic inositol 1,4,5-trisphosphate 5-phosphatase 1'}
{'amph_human': 'amph_human', 'amph': 'AMPH', 'amph1': 'AMPH1'}
{'synj1_human': 'synj1_human', 'synj1': 'SYNJ1', 'kiaa0910': 'KIAA0910', 'synaptic inositol 1,4,5-trisphosphate 5-phosphatase 1': 'Synaptic inositol 1,4,5-trisphosphate 5-phosphatase 1'}
{'amph_human': 'amph_human', 'amph': 'AMPH', 'amph1': 'AMPH1'}
{'synj1_human': 'synj1_human', 'synj1': 'SYNJ1', 'kiaa0910': 'KIAA0910', 'synaptic inositol 1,4,5-trisphosphate 5-phosphatase 1': 'Synaptic inositol 1,4,5-trisphosphate 5-phosphatase 1'}


In [51]:
interactorsA[0].get(k[0].lower())

In [52]:
k[0].lower()

'obesity'

In [87]:
experiments=[]

In [88]:
experiments=[]
for i in range(len(interactorsA)):
    if interactorsA[i].get('apoe') :
        experiments.append((interactorsA[i],interactorsB[i]))
        

In [89]:
len(experiments)

212

In [75]:
experiment[:10]

[({'apoe_human': 'apoe_human', 'apoe': 'APOE'},
  {'hpt_human': 'hpt_human', 'hp': 'HP', 'zonulin': 'Zonulin'}),
 ({'apoe_human': 'apoe_human', 'apoe': 'APOE'},
  {'hpt_human': 'hpt_human', 'hp': 'HP', 'zonulin': 'Zonulin'}),
 ({'apoe_human': 'apoe_human', 'apoe': 'APOE'},
  {'hpt_human': 'hpt_human', 'hp': 'HP', 'zonulin': 'Zonulin'}),
 ({'apoe_human': 'apoe_human', 'apoe': 'APOE'},
  {'hpt_human': 'hpt_human', 'hp': 'HP', 'zonulin': 'Zonulin'}),
 ({'apoe_human': 'apoe_human', 'apoe': 'APOE'},
  {'hpt_human': 'hpt_human', 'hp': 'HP', 'zonulin': 'Zonulin'}),
 ({'apoe_human': 'apoe_human', 'apoe': 'APOE'},
  {'pdcd4_human': 'pdcd4_human',
   'pdcd4': 'PDCD4',
   'h731': 'H731',
   'nuclear antigen h731-like': 'Nuclear antigen H731-like',
   'neoplastic transformation inhibitor protein': 'Neoplastic transformation inhibitor protein',
   'protein 197/15a': 'Protein 197/15a'}),
 ({'apoe_human': 'apoe_human', 'apoe': 'APOE'},
  {'cdc37_human': 'cdc37_human',
   'cdc37': 'CDC37',
   'hsp90 c

In [44]:
direction_list[:3]

[('TNXB', 'SKIV2L'), ('FKBPL', 'SKIV2L'), ('rs12153855', 'SKIV2L')]

In [138]:
mutations_intact=pd.read_csv('../intact/mutations.tsv', sep='\t')

/tmp/ipykernel_81816/2719859885.py:1: DtypeWarning: Columns (9,12) have mixed types. Specify dtype option on import or set low_memory=False.
  mutations_intact=pd.read_csv('../intact/mutations.tsv', sep='\t')


In [140]:
mutations_intact.head(10)

,#Feature AC,Feature short label,Feature range(s),Original sequence,Resulting sequence,Feature type,Feature annotation,Affected protein AC,Affected protein symbol,Affected protein full name,Affected protein organism,Interaction participants,PubMedID,Figure legend,Interaction AC
0,EBI-489661,P15153:p.Gln61Leu,61-61,Q,L,mutation(MI:0118),NaN,uniprotkb:P15153,RAC2,NaN,9606 - Homo sapiens,"uniprotkb:P15153(protein(MI:0326), 9606 - Homo sapiens)|ensembl:ENSP00000356505.4(protein(MI:0326), 9606 - Homo sapiens)",11090627,NaN,EBI-489644
1,EBI-496084,P42573:p.Gly360Ser,360-360,G,S,mutation(MI:0118),NaN,uniprotkb:P42573,ced-3,NaN,6239 - Caenorhabditis elegans,"uniprotkb:P30429(protein(MI:0326), 6239 - Caenorhabditis elegans)|uniprotkb:P42573(protein(MI:0326), 6239 - Caenorhabditis elegans)",9261161,NaN,EBI-495051
2,EBI-495357,Q99640:p.Asn238Ala,238-238,N,A,mutation(MI:0118),NaN,uniprotkb:Q99640,PKMYT1,NaN,9606 - Homo sapiens,"uniprotkb:Q99640(protein(MI:0326), 9606 - Homo sapiens)|uniprotkb:P06493(protein(MI:0326), 9606 - Homo sapiens)",10373560,Fig. 1B,EBI-495348
3,EBI-476918,Q14185:p.[Trp45Ala;Gly171Asp],45-45,W,A,mutation(MI:0118),NaN,uniprotkb:Q14185,DOCK1,NaN,9606 - Homo sapiens,"uniprotkb:Q92556(protein(MI:0326), 9606 - Homo sapiens)|uniprotkb:Q14185(protein(MI:0326), 9606 - Homo sapiens)|uniprotkb:P63000(protein(MI:0326), 9606 - Homo sapiens)",15247908,NaN,EBI-446978
4,EBI-476918,Q14185:p.[Trp45Ala;Gly171Asp],171-171,G,D,mutation(MI:0118),NaN,uniprotkb:Q14185,DOCK1,NaN,9606 - Homo sapiens,"uniprotkb:Q92556(protein(MI:0326), 9606 - Homo sapiens)|uniprotkb:Q14185(protein(MI:0326), 9606 - Homo sapiens)|uniprotkb:P63000(protein(MI:0326), 9606 - Homo sapiens)",15247908,NaN,EBI-446978
5,EBI-490500,P50402:p.[Glu145Ala;Glu146Ala],145-145,E,A,mutation(MI:0118),NaN,uniprotkb:P50402,EMD,NaN,9606 - Homo sapiens,"uniprotkb:Q9NYF8(protein(MI:0326), 9606 - Homo sapiens)|ensembl:ENSP00000358857.4(protein(MI:0326), 9606 - Homo sapiens)",15009215,NaN,EBI-490049
6,EBI-490500,P50402:p.[Glu145Ala;Glu146Ala],146-146,E,A,mutation(MI:0118),NaN,uniprotkb:P50402,EMD,NaN,9606 - Homo sapiens,"uniprotkb:Q9NYF8(protein(MI:0326), 9606 - Homo sapiens)|ensembl:ENSP00000358857.4(protein(MI:0326), 9606 - Homo sapiens)",15009215,NaN,EBI-490049
7,EBI-490607,P13423:p.Glu762Cys,762-762,E,C,mutation(MI:0118),NaN,uniprotkb:P13423,pagA,NaN,1392 - Bacillus anthracis,"uniprotkb:P13423(protein(MI:0326), 1392 - Bacillus anthracis)|uniprotkb:P13423(protein(MI:0326), 1392 - Bacillus anthracis)",15044490,NaN,EBI-457002
8,EBI-490557,P13423:p.Glu762Cys,762-762,E,C,mutation(MI:0118),NaN,uniprotkb:P13423,pagA,NaN,1392 - Bacillus anthracis,"uniprotkb:P58335(protein(MI:0326), 9606 - Homo sapiens)|uniprotkb:P13423(protein(MI:0326), 1392 - Bacillus anthracis)",15044490,NaN,EBI-456901
9,EBI-490572,P58335:p.Arg40Cys,40-40,R,C,mutation(MI:0118),NaN,uniprotkb:P58335,ANTXR2,NaN,9606 - Homo sapiens,"uniprotkb:P58335(protein(MI:0326), 9606 - Homo sapiens)|uniprotkb:P13423(protein(MI:0326), 1392 - Bacillus anthracis)",15044490,NaN,EBI-456911


In [29]:
def work(k,i,df_intact,experiment):
    for spl1 in df_intact.iloc[i,4].split('|'):
        for spl2 in df_intact.iloc[i,5].split('|'):
            if re.search(r'(.*:)(.*)(\(.*\))',spl1).groups()[1]==k[0] and \
            re.search(r'(.*:)(.*)(\(.*\))',spl2).groups()[1]==k[1]:
                experiment.append((k[0],k[1]))
                break
        else:
            continue
        break
if __name__=='__main__':
    start=datetime.datetime.now()
    print('Parent process %s.' % os.getpid())
    p=Pool(60)
    manager=Manager()
    experiment=manager.list()
    for k in direction_list:
        for i in range(len(df_intact)):
            p.apply_async(work, args=(k,i,df_intact,experiment))
    print('Waiting for all subprocesses done...')
    
    p.close()
    p.join()
    print('All subprocesses done.')

    end=datetime.datetime.now()
    print("The running time is "+str((end-start).seconds)+"s")  

Parent process 81816.


Process ForkPoolWorker-54:
Process ForkPoolWorker-55:
Process ForkPoolWorker-58:
Process ForkPoolWorker-48:
Process ForkPoolWorker-59:
Process ForkPoolWorker-60:
Process ForkPoolWorker-49:
Process ForkPoolWorker-41:
Process ForkPoolWorker-30:
Process ForkPoolWorker-38:
Process ForkPoolWorker-32:
Process ForkPoolWorker-23:
Process ForkPoolWorker-25:
Process ForkPoolWorker-33:
Process ForkPoolWorker-50:
Process ForkPoolWorker-46:


KeyboardInterrupt: 

Process ForkPoolWorker-47:
Process ForkPoolWorker-43:
Process ForkPoolWorker-13:
Process ForkPoolWorker-44:
Process ForkPoolWorker-56:
Process ForkPoolWorker-6:
Process ForkPoolWorker-5:
Process ForkPoolWorker-29:
Process ForkPoolWorker-45:
Process ForkPoolWorker-22:
Process ForkPoolWorker-31:
Process ForkPoolWorker-18:
Process ForkPoolWorker-12:
Process ForkPoolWorker-8:
Process ForkPoolWorker-1:
Process ForkPoolWorker-53:
Process ForkPoolWorker-34:
Process ForkPoolWorker-42:
Process ForkPoolWorker-51:
Process ForkPoolWorker-57:
Process ForkPoolWorker-52:
Process ForkPoolWorker-11:
Process ForkPoolWorker-37:
Process ForkPoolWorker-14:
Process ForkPoolWorker-2:
Process ForkPoolWorker-21:
Process ForkPoolWorker-3:
Process ForkPoolWorker-7:
Process ForkPoolWorker-10:
Process ForkPoolWorker-20:
Process ForkPoolWorker-19:
Process ForkPoolWorker-24:
Process ForkPoolWorker-15:
Process ForkPoolWorker-35:
Process ForkPoolWorker-17:
Process ForkPoolWorker-16:
Process ForkPoolWorker-28:
Process 

In [30]:
len(list(experiment))

0

Process ForkPoolWorker-103:
Traceback (most recent call last):
  File "/root/miniconda3/envs/hald/lib/python3.8/multiprocessing/process.py", line 315, in _bootstrap
    self.run()
  File "/root/miniconda3/envs/hald/lib/python3.8/multiprocessing/process.py", line 108, in run
    self._target(*self._args, **self._kwargs)
  File "/root/miniconda3/envs/hald/lib/python3.8/multiprocessing/pool.py", line 114, in worker
    task = get()
  File "/root/miniconda3/envs/hald/lib/python3.8/multiprocessing/queues.py", line 355, in get
    with self._rlock:
  File "/root/miniconda3/envs/hald/lib/python3.8/multiprocessing/synchronize.py", line 95, in __enter__
    return self._semlock.__enter__()
KeyboardInterrupt


In [172]:
k

('TNXB', 'SKIV2L')

In [166]:
df_intact.iloc[i,4]

'uniprotkb:NFKBIA(gene name)|psi-mi:NFKBIA(display_short)|uniprotkb:IKBA(gene name synonym)|uniprotkb:MAD3(gene name synonym)|uniprotkb:NFKBI(gene name synonym)|intact:I-kappa-B-alpha(synonym)|intact:Major histocompatibility complex enhancer-binding protein MAD3(synonym)|ensembl:ikba_human_gene(shortlabel)|psi-mi:ENSG00000100906(display_long)'

In [157]:
re.search(r'(psi-mi:)(.*)(\(display_long\))',file.iloc[i,4].split('|')[0])

In [103]:
xxxx=[]
for i in Relation_raw_all:
    if i.get('openie') and i.get('networkx') and i.get('networkx')[0]!=i.get('target1'):
        xxxx.append(i)

In [104]:
len(xxxx)

401

In [105]:
xxxx[:3]

[{'_id': ObjectId('63988454c88002a2f1f375b0'),
  'PMID': '12849688',
  'entity1': 'NAGLU',
  'entity2': 'Fanconi Syndrome',
  'sentence': 'Among the four tubular dysfunction markers, NAG showed the closest correlation with Cd, followed by alpha(1)-MG and then beta(2)-MG, and RBP was least so although the correlations were all statistically significant.',
  'AB': 'Comparative evaluation was made on alpha(1)-microglobulin (alpha(1)-MG), beta(2)-microglobulin (beta(2)-MG), retinol binding protein (RBP) and N-acetyl-beta-D-glucosaminidase (NAG), as a marker of renal tubular dysfunction after environmental exposure to cadmium (Cd), with special references to the effects of aging and correction for creatinine concentration. For this purpose, a previously established database of 817 never-smoking Japanese women (at the ages of 20 to 74 years) on hematological [hemoglobin, serum ferritin (FE), etc.] and urinary parameters [alpha(1)-MG, beta(2)-MG, creatinine (cr), and a specific gravity] was r

In [86]:
xxx=[]
for i in Relation_raw_all:
    if i.get('openie') and not i.get('networkx'):
        xxx.append(i)

In [87]:
len(xxx)

2385

In [88]:
xxx[:3]

[{'_id': ObjectId('63987fb6c88002a2f1f125ae'),
  'PMID': '19668188',
  'entity1': 'CDKN2A',
  'entity2': 'CDKN2B',
  'sentence': 'The Ink4/Arf locus comprises the Cdkn2a-Cdkn2b genes encoding three potent tumour suppressors, namely p16(Ink4a), p19(Arf) and p15(Ink4b), which are basally expressed in differentiated cells and upregulated by aberrant mitogenic signals.',
  'AB': "The mechanisms involved in the reprogramming of differentiated cells into induced pluripotent stem (iPS) cells by the three transcription factors Oct4 (also known as Pou5f1), Klf4 and Sox2 remain poorly understood. The Ink4/Arf locus comprises the Cdkn2a-Cdkn2b genes encoding three potent tumour suppressors, namely p16(Ink4a), p19(Arf) and p15(Ink4b), which are basally expressed in differentiated cells and upregulated by aberrant mitogenic signals. Here we show that the locus is completely silenced in iPS cells, as well as in embryonic stem (ES) cells, acquiring the epigenetic marks of a bivalent chromatin domain,

In [295]:
for i,j in relation_dict_copy.items():
    if j[0].get('PMID')=='23396152':
        print(j[0].get('sentence'))

In [296]:
for i in triple:
    if i.get('PMID')=='23396152':
        print(i.get('sentence'))

NameError: name 'triple' is not defined

In [86]:
xxxx=[]
for i in Relation_raw_all:
    if i.get('allennlp_final') and i.get('networkx') and i.get('allennlp_final')[0][1]!= i.get('networkx')[1]:
        xxxx.append(i)

In [87]:
len(xxxx)

6495

In [90]:
xxxx[-10:-1]

[{'PMID': '33247391',
  'entity1': 'Carcinogenesis',
  'entity2': 'Checkpoint Kinase 1',
  'sentence': 'INTRODUCTION: In September 2019, ranitidine and nizatidine were suggested to contain N-nitrosodimethylamine, a carcinogenic substance.',
  'AB': 'INTRODUCTION: In September 2019, ranitidine and nizatidine were suggested to contain N-nitrosodimethylamine, a carcinogenic substance. People have since been concerned about the potential impact of ranitidine/nizatidine use on the risk of cancer. OBJECTIVE: The objective of this study was to investigate the risk of cancer among people receiving ranitidine or nizatidine compared with other histamine 2 receptor antagonists (H2 blockers) [cimetidine, famotidine, roxatidine, and lafutidine]. METHODS: In the Japan Medical Data Center claims database (comprising people aged < 75 years) from 2005 to 2018, we identified new adult users of H2 blockers and classified them into ranitidine/nizatidine users and other H2 blocker users. We estimated the i